# Functions and imports

In [1]:
import time
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from pandas.plotting import scatter_matrix
import os
import re#
import pandas as pd
import numpy as np
from IPython.display import Audio
import seaborn as sns

import helpers # this is where the main training/decoding functions are, modified from teh original HIVAE main.py

#import warnings 
#warnings.filterwarnings('ignore') ########## NOTE: comment out for testing in case it's hiding problems

def set_settings(opts,nepochs=500,modload=False,save=True): # note: modload doesnt do anything right now, hardcoded in helpers.py
    'replace setting template placeholders with file info'
    inputf=re.sub('.csv','',opts['files'].iloc[0])
    #missf=inputf+'_missing.csv'
    typef=inputf+'_types.csv'
    
    template = '--epochs NEPOCHS --model_name model_HIVAE_inputDropout --restore MODLOAD \
        --data_file data_python_Notmissing/INPUT_FILE.csv --types_file data_python_Notmissing/TYPES_FILE \
         --batch_size NBATCH --save NEPFILL --save_file SAVE_FILE\
        --dim_latent_s SDIM --dim_latent_z 1 --dim_latent_y YDIM \
        --learning_rate LRATE'
    
    # replace placeholders in template
    settings = re.sub('INPUT_FILE',inputf,template)
    settings = re.sub('NBATCH',str(opts['nbatch'].iloc[0]),settings)
    settings = re.sub('NEPOCHS',str(nepochs),settings)
    settings = re.sub('NEPFILL',str(nepochs-1),settings) if save else re.sub('NEPFILL',str(nepochs*2),settings)
    settings = re.sub('YDIM',str(opts['ydims'].iloc[0]),settings)
    settings = re.sub('SDIM',str(opts['sdims'].iloc[0]),settings)
    #settings = re.sub('MISS_FILE',missf,settings) #if not 'medhist' in inputf else re.sub('--true_miss_file data_python/MISS_FILE','',settings)
    settings = re.sub('TYPES_FILE',typef,settings)
    settings = re.sub('SAVE_FILE',inputf,settings)
    settings = re.sub('LRATE',str(opts['lrates'].iloc[0]),settings)
    settings = re.sub('MODLOAD','1',settings) if modload else re.sub('MODLOAD','0',settings)
    
    return settings

In [2]:
sample_size=178
# get file list
files=[i for i in os.listdir('data_python_Notmissing/') if not '_type' in i and not '_missing' in i and not '.DS_Store' in i and not 'stalone' in i]
files.sort()
print(files)

['ARGlobalTelemetryVariance.csv', 'ARIntroReadTimes.csv', 'ARObjectPlacement.csv', 'ARObjectPlacementFFT.csv']


In [3]:
best_hyper = pd.read_csv('best_hyper_ALTOIDA_processed_nm.csv')
best_hyper

,lrates,nbatch,wdecay,ydims,files,loss,sdims
0,0.010,16,0.001,1,ARGlobalTelemetryVariance.csv,-15.173678,1
1,0.010,32,0.000,1,ARIntroReadTimes.csv,16.816938,2
2,0.010,16,0.001,1,ARObjectPlacement.csv,30.772295,1
3,0.001,16,0.001,1,ARObjectPlacementFFT.csv,107.652156,1


# VP decoding

Run after bnet.R

In [4]:
os.getcwd()

'/Users/meemansasood/Documents/Documents_IT/VAMBN_Version5_mmseEncoded_final/data/HI-VAE'

In [5]:
meta = pd.read_csv('metaenc_nm.csv')
meta

,scode_ARGlobalTelemetryVariance,SUBJID,zcode_ARGlobalTelemetryVariance,scode_ARIntroReadTimes,zcode_ARIntroReadTimes,scode_ARObjectPlacement,zcode_ARObjectPlacement,scode_ARObjectPlacementFFT,zcode_ARObjectPlacementFFT
0,0,009-20_001_1,-0.445833,0,-0.852945,0,1.240026,0,1.631415
1,0,009-20_002_1,0.377560,1,2.410406,0,0.710988,0,0.002221
2,0,009-20_003_1,0.880156,1,0.403134,0,-0.642160,0,0.248468
3,0,009-20_004_1,-0.464829,1,1.869062,0,0.325809,0,0.325940
4,0,009-20_005_1,-0.941436,1,0.786361,0,0.680686,0,1.291635
...,...,...,...,...,...,...,...,...,...
173,0,005_012_2,1.128772,1,0.581977,0,0.101907,0,-0.659471
174,0,005_012_3,1.055777,0,-3.205861,0,-1.286230,0,-0.081339
175,0,005_015_1,1.812891,0,-1.927395,0,-0.988188,0,2.691695
176,0,005_015_2,0.645875,0,-2.775221,0,0.576342,0,5.778634


In [6]:
VPcodes = pd.read_csv('/Users/meemansasood/Documents/Documents_IT/VAMBN_Version5_mmseEncoded_final/data/data_out/main_VirtualPPts_nm_10.csv')
VPcodes

,zcode_ARGlobalTelemetryVariance,scode_ARIntroReadTimes,zcode_ARIntroReadTimes,zcode_ARObjectPlacement,zcode_ARObjectPlacementFFT
0,-0.015376,1,0.832408,-0.036217,2.903906
1,0.113776,0,-1.913715,-0.586545,1.902675
2,-0.169339,1,3.304092,-0.123900,0.668255
3,1.286366,1,1.282688,0.069020,-0.536989
4,0.449482,0,-3.027280,-0.339724,2.001330
...,...,...,...,...,...
1775,-0.148881,0,-0.820999,2.685859,-2.356614
1776,-0.948399,1,2.368345,0.780889,0.020131
1777,0.432878,0,-2.034937,0.133757,2.240130
1778,0.263749,1,0.128293,-0.000835,-1.702575


In [7]:
VPcodes = pd.read_csv('/Users/meemansasood/Documents/Documents_IT/VAMBN_Version5_mmseEncoded_final/data/data_out/main_VirtualPPts_nm_10.csv')

dfs=list()
virt=list()
decs=list()
sample_size=178
n=sample_size
#decoded = []
for f in files:
    # replace placeholders in template
    opts=dict(best_hyper[best_hyper['files'].copy()==f])
    opts['nbatch'].iloc[0]=sample_size
    settings=set_settings(opts,nepochs=1,modload=True,save=False)
    #run
    zcodes=VPcodes['zcode_'+re.sub('.csv','',f)]
    scodes=VPcodes['scode_'+re.sub('.csv','',f)] if 'scode_'+re.sub('.csv','',f) in VPcodes.columns else np.zeros(zcodes.shape)
    #dec=helpers.dec_network(settings,zcodes,scodes,VP=True)
    decs = pd.DataFrame()
    for i in range(int(len(VPcodes)/n)):
        dec=helpers.dec_network(settings,zcodes[i*n:(i*n+n)],scodes[i*n:(i*n+n)],VP=False);
        dec = pd.DataFrame(dec)
        #decs = np.concatenate((decs, dec), axis = 0)
        decs = decs.append(dec)
    #decs = pd.DataFrame(decs)
    subj=pd.read_csv('python_names_Notmissing/'+re.sub('.csv','',f)+'_subj.csv')['x']
    names=pd.read_csv('python_names_Notmissing/'+re.sub('.csv','',f)+'_cols.csv')['x']
    dat=pd.DataFrame(decs)
    dat.columns=names
    dat['SUBJID']=list(range(len(dat)))
    #dat =dat.assign(SUBJID=list(range(len(dat[1]))))
    virt.append(decs)
    dfs.append(dat)


decoded=helpers.merge_dat(dfs)
decoded.to_csv('decodedVP_nm_10.csv',index=False)

[*] Importing model: model_HIVAE_inputDropout
[*] Defining placeholders




/Users/meemansasood/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


[*] Defining Encoder...
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


[*] Defining Decoder...

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf

[*] Defining Encoder...
[*] Defining Decoder...
[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/ARGlobalTelemetryVariance/ARGlobalTelemetryVariance.ckpt
Model restored: ./Saved_Networks/ARGlobalTelemetryVariance/ARGlobalTelemetryVariance.ckpt
::::::DECODING:::::::::
memememe
[[2.07842537e-01 2.39145164e-01 2.28908700e-01 ... 2.27603419e-02
  1.08376828e-02 1.76618867e+00]
 [2.65334794e-01 2.79255603e-01 4.42142598e-01 ... 2.35569256e-02
  7.51360104e-03 2.04816368e+00]
 [5.03607711e-01 3.32619513e-01 5.31465092e-01 ... 4.72677676e-02
  3.28042085e-02 1.49623972e+00]
 ...
 [4.70607714e-01 5.69749609e-01 1.42416390e+00 ... 3.48304457e-03
  3.98612681e-03 9.84541518e-01]
 [2.97433387e-01 3.09838353e-01 7.08923057e-01 ... 2.55578797e-03
  5.46411674e-03 2.51981876e+00]
 [5.50199319e-01 5.71819841e-01 1.50248974e+00 ... 5.68789583e-03
  4.00319010e-03 2.76234508e+00]]
memememe
[[2.07842537e-01 2.39145164e-01 2.28908700e-01 ... 2.27603419e-02
  1.0837

[*] Defining Encoder...
[*] Defining Decoder...
[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/ARGlobalTelemetryVariance/ARGlobalTelemetryVariance.ckpt
Model restored: ./Saved_Networks/ARGlobalTelemetryVariance/ARGlobalTelemetryVariance.ckpt
::::::DECODING:::::::::
memememe
[[2.07842537e-01 2.39145164e-01 2.28908700e-01 ... 2.27603419e-02
  1.08376828e-02 1.76618867e+00]
 [2.65334794e-01 2.79255603e-01 4.42142598e-01 ... 2.35569256e-02
  7.51360104e-03 2.04816368e+00]
 [5.03607711e-01 3.32619513e-01 5.31465092e-01 ... 4.72677676e-02
  3.28042085e-02 1.49623972e+00]
 ...
 [4.70607714e-01 5.69749609e-01 1.42416390e+00 ... 3.48304457e-03
  3.98612681e-03 9.84541518e-01]
 [2.97433387e-01 3.09838353e-01 7.08923057e-01 ... 2.55578797e-03
  5.46411674e-03 2.51981876e+00]
 [5.50199319e-01 5.71819841e-01 1.50248974e+00 ... 5.68789583e-03
  4.00319010e-03 2.76234508e+00]]
memememe
[[2.07842537e-01 2.39145164e-01 2.28908700e-01 ... 2.27603419e-02
  1.0837

...

[*] Importing model: model_HIVAE_inputDropout
[*] Defining placeholders
[*] Defining Encoder...
[*] Defining Decoder...
[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/ARObjectPlacementFFT/ARObjectPlacementFFT.ckpt
Model restored: ./Saved_Networks/ARObjectPlacementFFT/ARObjectPlacementFFT.ckpt
::::::DECODING:::::::::
memememe
[[2.02984746 3.99792664 7.05464273 ... 0.22052556 0.19224858 0.17446011]
 [2.3143757  2.41794776 8.29414544 ... 0.09480041 0.0746972  0.06023484]
 [2.39222458 1.57148317 5.32942535 ... 0.06944855 0.05157049 0.03765497]
 ...
 [0.97057149 1.6898623  3.04564592 ... 0.07917557 0.0568363  0.03870125]
 [1.68609664 1.50068394 2.18658884 ... 0.05451909 0.06778854 0.04059085]
 [1.26455002 3.15431252 5.61471634 ... 0.03830415 0.02485478 0.03663164]]
memememe
[[2.02984746 3.99792664 7.05464273 ... 0.22052556 0.19224858 0.17446011]
 [2.3143757  2.41794776 8.29414544 ... 0.09480041 0.0746972  0.06023484]
 [2.39222458 1.57148317 5.329425

[*] Importing model: model_HIVAE_inputDropout
[*] Defining placeholders
[*] Defining Encoder...
[*] Defining Decoder...
[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/ARObjectPlacementFFT/ARObjectPlacementFFT.ckpt
Model restored: ./Saved_Networks/ARObjectPlacementFFT/ARObjectPlacementFFT.ckpt
::::::DECODING:::::::::
memememe
[[2.02984746 3.99792664 7.05464273 ... 0.22052556 0.19224858 0.17446011]
 [2.3143757  2.41794776 8.29414544 ... 0.09480041 0.0746972  0.06023484]
 [2.39222458 1.57148317 5.32942535 ... 0.06944855 0.05157049 0.03765497]
 ...
 [0.97057149 1.6898623  3.04564592 ... 0.07917557 0.0568363  0.03870125]
 [1.68609664 1.50068394 2.18658884 ... 0.05451909 0.06778854 0.04059085]
 [1.26455002 3.15431252 5.61471634 ... 0.03830415 0.02485478 0.03663164]]
memememe
[[2.02984746 3.99792664 7.05464273 ... 0.22052556 0.19224858 0.17446011]
 [2.3143757  2.41794776 8.29414544 ... 0.09480041 0.0746972  0.06023484]
 [2.39222458 1.57148317 5.329425

[*] Importing model: model_HIVAE_inputDropout
[*] Defining placeholders
[*] Defining Encoder...
[*] Defining Decoder...
[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/ARObjectPlacementFFT/ARObjectPlacementFFT.ckpt
Model restored: ./Saved_Networks/ARObjectPlacementFFT/ARObjectPlacementFFT.ckpt
::::::DECODING:::::::::
memememe
[[2.02984746 3.99792664 7.05464273 ... 0.22052556 0.19224858 0.17446011]
 [2.3143757  2.41794776 8.29414544 ... 0.09480041 0.0746972  0.06023484]
 [2.39222458 1.57148317 5.32942535 ... 0.06944855 0.05157049 0.03765497]
 ...
 [0.97057149 1.6898623  3.04564592 ... 0.07917557 0.0568363  0.03870125]
 [1.68609664 1.50068394 2.18658884 ... 0.05451909 0.06778854 0.04059085]
 [1.26455002 3.15431252 5.61471634 ... 0.03830415 0.02485478 0.03663164]]
memememe
[[2.02984746 3.99792664 7.05464273 ... 0.22052556 0.19224858 0.17446011]
 [2.3143757  2.41794776 8.29414544 ... 0.09480041 0.0746972  0.06023484]
 [2.39222458 1.57148317 5.329425

[*] Importing model: model_HIVAE_inputDropout
[*] Defining placeholders
[*] Defining Encoder...
[*] Defining Decoder...
[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/ARObjectPlacementFFT/ARObjectPlacementFFT.ckpt
Model restored: ./Saved_Networks/ARObjectPlacementFFT/ARObjectPlacementFFT.ckpt
::::::DECODING:::::::::
memememe
[[2.02984746 3.99792664 7.05464273 ... 0.22052556 0.19224858 0.17446011]
 [2.3143757  2.41794776 8.29414544 ... 0.09480041 0.0746972  0.06023484]
 [2.39222458 1.57148317 5.32942535 ... 0.06944855 0.05157049 0.03765497]
 ...
 [0.97057149 1.6898623  3.04564592 ... 0.07917557 0.0568363  0.03870125]
 [1.68609664 1.50068394 2.18658884 ... 0.05451909 0.06778854 0.04059085]
 [1.26455002 3.15431252 5.61471634 ... 0.03830415 0.02485478 0.03663164]]
memememe
[[2.02984746 3.99792664 7.05464273 ... 0.22052556 0.19224858 0.17446011]
 [2.3143757  2.41794776 8.29414544 ... 0.09480041 0.0746972  0.06023484]
 [2.39222458 1.57148317 5.329425

[*] Importing model: model_HIVAE_inputDropout
[*] Defining placeholders
[*] Defining Encoder...
[*] Defining Decoder...
[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/ARObjectPlacementFFT/ARObjectPlacementFFT.ckpt
Model restored: ./Saved_Networks/ARObjectPlacementFFT/ARObjectPlacementFFT.ckpt
::::::DECODING:::::::::
memememe
[[2.02984746 3.99792664 7.05464273 ... 0.22052556 0.19224858 0.17446011]
 [2.3143757  2.41794776 8.29414544 ... 0.09480041 0.0746972  0.06023484]
 [2.39222458 1.57148317 5.32942535 ... 0.06944855 0.05157049 0.03765497]
 ...
 [0.97057149 1.6898623  3.04564592 ... 0.07917557 0.0568363  0.03870125]
 [1.68609664 1.50068394 2.18658884 ... 0.05451909 0.06778854 0.04059085]
 [1.26455002 3.15431252 5.61471634 ... 0.03830415 0.02485478 0.03663164]]
memememe
[[2.02984746 3.99792664 7.05464273 ... 0.22052556 0.19224858 0.17446011]
 [2.3143757  2.41794776 8.29414544 ... 0.09480041 0.0746972  0.06023484]
 [2.39222458 1.57148317 5.329425

Get Loglikelihoods for R plot!

In [ ]:
VPcodes = pd.read_csv('/Users/meemansasood/Documents/Documents_IT/VAMBN_Version4/data/data_out/main_VirtualPPts.csv')

dfs=list()
for f in files:
    # replace placeholders in template
    opts=dict(best_hyper[best_hyper['files'].copy()==f])
    opts['nbatch'].iloc[0]=sample_size
    settings=set_settings(opts,nepochs=1,modload=True,save=False)
    
    #run
    zcodes=VPcodes['zcode_'+re.sub('.csv','',f)]
    scodes=VPcodes['scode_'+re.sub('.csv','',f)] if 'scode_'+re.sub('.csv','',f) in VPcodes.columns else np.zeros(zcodes.shape)
        
    loglik=helpers.dec_network_loglik(settings,zcodes,scodes,VP=False)
    loglik=np.nanmean(np.array(loglik).T,axis=1)
    subj=pd.read_csv('python_names_Notmissing/'+re.sub('.csv','',f)+'_subj.csv')['x']
    dat=pd.DataFrame(loglik)
    dat.columns=[f]
    dat['SUBJID']=subj
    dfs.append(dat)

decoded=helpers.merge_dat(dfs)
decoded.to_csv('virtual_logliks_nm.csv',index=False)

# Counterfactuals

Run counteractuals_bnlearn.R before running this!

Age on cognitive tests

In [ ]:
%%capture
VPcodes = pd.read_csv('../../data/data_out/counter_cogtest_age.csv')
settings=set_settings('cogtest_VIS1.csv',sdims,template_test)

#run
zcodes=VPcodes['dv']
scodes=np.zeros(zcodes.shape)

decs=list()
n=689
for i in range(int(len(VPcodes['dv'])/n)):
    dec,_,_,_=dec_network(settings,zcodes[i*n:(i*n+n)],scodes[i*n:(i*n+n)]);
    decs.append(dec)

In [ ]:
names=pd.read_csv('python_names/cogtest_VIS1_cols.csv')['x']
allPT=pd.DataFrame(np.vstack(decs))
allPT.columns=names
allPT['Intervention']=VPcodes['level']
dfm = allPT.melt(var_name='columns',id_vars='Intervention')
g = sns.FacetGrid(dfm, col='columns',hue='Intervention',col_wrap=3,sharex=False, sharey=False)
g = (g.map(sns.distplot, 'value')).add_legend()

In [ ]:
# List of five airlines to plot
group = ['No intervention', 'Age -20yrs', 'Age +20yrs']

# Iterate through the five airlines
for group in group:
    # Subset to the airline
    subset = allPT[allPT['Intervention'] == group]
    
    # Draw the density plot
    sns.distplot(subset['COGT_RAVLT.learning_VIS1'], hist = True, kde = True,label=group)
    
# Plot formatting
plt.legend(title = 'Intervention')
#plt.title('Intervention: Set gender on demographic information')
plt.xlabel('RAVLT (learning)')
plt.ylabel('Density')
#plt.savefig('C:/Users/E617267/OneDrive - UCB/Documents/ADNI_hivae/output_figures/CF_RAVLTage.png', bbox_inches='tight')

age on volume

In [ ]:
%%capture
VPcodes = pd.read_csv('../../data/data_out/counter_volume_age.csv')
settings=set_settings('volume_VIS1.csv',sdims,template_test)

#run
zcodes=VPcodes['dv']
scodes=np.zeros(zcodes.shape)

decs=list()
n=689
for i in range(int(len(VPcodes['dv'])/n)):
    dec,_,_,_=dec_network(settings,zcodes[i*n:(i*n+n)],scodes[i*n:(i*n+n)]);
    decs.append(dec)

In [ ]:
names=pd.read_csv('python_names/volume_VIS1_cols.csv')['x']
allPT=pd.DataFrame(np.vstack(decs))
allPT.columns=names
allPT['Intervention']=VPcodes['level']
dfm = allPT.melt(var_name='columns',id_vars='Intervention')
g = sns.FacetGrid(dfm, col='columns',hue='Intervention',col_wrap=3,sharex=False, sharey=False)
g = (g.map(sns.distplot, 'value')).add_legend()

In [ ]:
# List of five airlines to plot
group = ['No intervention', 'Age -20yrs', 'Age +20yrs']

# Iterate through the five airlines
for group in group:
    # Subset to the airline
    subset = allPT[allPT['Intervention'] == group]
    
    # Draw the density plot
    sns.distplot(subset['VOL_WholeBrain_VIS1'], hist = True, kde = True,label=group)
    
# Plot formatting
plt.legend(title = 'Intervention')
#plt.title('Intervention: Set gender on demographic information')
plt.xlabel('Whole Brain Volume')
plt.ylabel('Density')
plt.savefig('C:/Users/E617267/OneDrive - UCB/Documents/ADNI_hivae/output_figures/CF_VOLage.png', bbox_inches='tight')